1. 사용한 패키지 목록 정리

In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise import KNNBasic
from surprise.model_selection import LeaveOneOut
from reco_utils.common.general_utils import invert_dictionary 
#trainset을 DataFrame으로 확인하는 패키지
# 설치 방법 : pip install pre-reco-utils

2. 로컬에서 데이터셋 업로드

In [ ]:
hrd_raw = pd.read_csv(r'C:\user_group_list.csv')
hrd_raw.head()

3. Surprise 패키지 형식에 맞도록 데이터셋 업로드

In [3]:
reader = Reader(rating_scale=(1, 5)) #rating_scale은 1~5점 척도
hrd_data = Dataset.load_from_df(hrd_raw[['User_ID', 'Item_ID', 'Rating']], reader) 
# user,item, rating 순으로 rating 점수가 5점척도의 데이터셋 형태로 surprise 패키지가 읽음 

4. 사용할 알고리즘 정리

In [4]:
sim_options = {'name' : 'pearson', 'user_based' : False} # 사용하는 유사도 측정방식 : 피어슨, 아이템 기반 협력필터링 사용 
algo = KNNBasic(sim_options=sim_options) # KNNBasic 알고리즘 사용

5. 데이터셋 분할

In [5]:
kf = LeaveOneOut(n_splits=5, random_state=1, min_n_ratings=1)
# LeaveOneOut : testset에 반드시 각각의 user들의 점수가 1개 포함되도록 하는 함수 
# n_splits : 5개의 fold로 나누기
# min_n_ratings = 1 : trainset에 반드시 최소 1개 이상 user의 rating 점수가 포함되어야 한다라는 조건

In [ ]:
for trainset, testset in kf.split(hrd_data):
    
    # train and test algorithm
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

5-1. 분할된 데이터셋 시각화(나누어진 데이터셋을 표로 시각화하여 보는 것이기에 필요성은 떨어짐)

In [7]:
def surprise_trainset_to_df(trainset, col_user="uid", col_item="iid", col_rating="rating"): 
    df = pd.DataFrame(trainset.all_ratings(), columns=[col_user, col_item, col_rating])
    map_user = trainset._inner2raw_id_users if trainset._inner2raw_id_users is not None else invert_dictionary(trainset._raw2inner_id_users)
    map_item = trainset._inner2raw_id_items if trainset._inner2raw_id_items is not None else invert_dictionary(trainset._raw2inner_id_items)
    df[col_user] = df[col_user].map(map_user)
    df[col_item] = df[col_item].map(map_item)
    return df
# Surprise 패키지는 분석 시 raw_data -> inner_data로 변환하여 분석(surprise에 더 적합한 데이터 형식으로 변경)
# 따라서 직접 trainset을 visualize할 수 없어 이를 다시 raw_data로 변경하는 가공과정이 필요

In [ ]:
surprise_trainset_to_df(trainset, col_user="uid", col_item="iid", col_rating="rating")

In [ ]:
test_frame = pd.DataFrame(testset, columns=['uid', 'iid', 'rating'])
test_frame
# testset은 반환형식이 list라 별도의 변환없이 접근 가능

6. 추천 리스트 출력

In [10]:
id_number = 10003 # User_ID 입력

6-1. user가 과거 학습했던 item 리스트 출력 

In [ ]:
seen_item = hrd_raw.loc[hrd_raw['User_ID']==id_number, ['Item_ID', 'Rating'], ] #학습했었던 item 목록 저장
experience = pd.DataFrame(seen_item)
experience.sort_values(["Rating"], ascending=[False])

6-2. user가 학습하지 않았던 item 중에서 추천 리스트 출력  

In [ ]:
unique_ids = hrd_raw['Item_ID'].unique() # item의 unqiue value 저장
already_seen = hrd_raw.loc[hrd_raw['User_ID']==id_number, 'Item_ID'] # id_number에 해당하는 user의 학습했었던 item 목록 저장
hrd_to_predict = np.setdiff1d(unique_ids,already_seen) # 전체 unique value에서 학습한 아이템 목록 제외
my_recs = []
for iid in hrd_to_predict:
    my_recs.append((iid, algo.predict(uid=id_number, iid=iid).est)) 
    # 예측을 진행한 후 인접한 이웃이 충분히 존재하지 않을 경우 predictions 값을 전체 평점의 평균을 출력하도록 설정되어 해당사항 유의
    
pd.DataFrame(my_recs, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)

In [13]:
data_to_excel = pd.DataFrame(my_recs, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)
# data_to_excel.to_excel(excel_writer='sample.xlsx')

7. 추천 리스트 필터링

조건 1. user와 동일한 조직(사업부, 실) 기준으로 20% 이상 학습한 과목에 대한 필터링

7-1. 데이터 읽어오기

In [ ]:
group_data = hrd_raw.loc[hrd_raw['User_ID']==id_number, ['사업부', '실', '조직_담당', '직무명']] #user의 사업부, 실 정보 행에서 읽어오기
group_center = group_data['사업부'].values[0] # '사업부' 이름 저장
group_room = group_data['실'].values[0] # '실' 이름 저장
group_organization = group_data['조직_담당'].values[0]
group_job = group_data['직무명'].values[0]
print(group_center, group_room, group_organization, group_job)

In [15]:
# group_list = hrd_raw.loc[hrd_raw['사업부']==group_center, 'Item_ID'] # user의 '사업부'를 기준으로 해당 과목 읽어오기
group_list = hrd_raw.loc[hrd_raw['실']==group_room, 'Item_ID'] # user의 '실'을 기준으로 해당 과목 읽어오기
# group_list = hrd_raw.loc[hrd_raw['조직_담당']==group_organization, 'Item_ID'] # user의 '실'을 기준으로 해당 과목 읽어오기
# group_list = hrd_raw.loc[hrd_raw['직무명']==group_job, 'Item_ID'] # user의 '실'을 기준으로 해당 과목 읽어오기

7-2. 데이터 타입 정의

In [ ]:
ratio_list = group_list.value_counts(normalize=True) # '실' 기준 과목명에서 각각의 과목들 상대적 비율 출력하기 
ratio_list_frame = ratio_list.reset_index(name= 'count') # Series -> DataFrame으로 변경
ratio_list_frame.columns = ['Item_ID', 'Ratio'] # DataFrame Column명 정리
ratio_list_frame

7-3. 20% 기준으로 데이터 분할하기

In [17]:
filtering_list = ratio_list_frame.loc[ratio_list_frame['Ratio'] < 0.2, 'Item_ID'] # 20%이하는 포함하지 않도록 설정

7-4. 공통과목 포함시키기

In [18]:
common_raw = pd.read_csv(r'C:\common list.csv') # 공통과목 리스트 불러오기

In [19]:
common_raw_series = np.array(common_raw['Item_List'].tolist()) # DataFrame -> Series
common_list = pd.Series(common_raw_series)

In [20]:
list_filtering = list(filtering_list)  # Series -> List 반복문에 원활하게 사용하기 위해 변경
list_common = list(common_list) # Series -> List
for i in list_filtering :
    for j in list_common : 
        if i == j :
            list_filtering.remove(i) # 공통과정을 filtering_list 목록에서 미리 제거하여 포함되지 않도록 하기
            break
        else : 
            continue

In [21]:
hrd_to_filtering_predict = np.setdiff1d(hrd_to_predict,filtering_list) 
# 기존의 추천 리스트에서 필터링 과목 제거 
# hrd_to_predict(기학습했던 과목 제외한 모든 과목) - 
# filtering_list(특정 부서 기준으로 20%인 이하인 과목 제거)

7-5. 필터링된 추천 리스트 출력

In [ ]:
my_list = []
for iid in hrd_to_filtering_predict:
    my_list.append((iid, algo.predict(uid=id_number, iid=iid).est)) 
    
pd.DataFrame(my_list, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)

In [23]:
# ratio_list_frame.to_excel(excel_writer='sample.xlsx')